In [4]:
from datasets import load_dataset
from datasets import get_dataset_config_names
import pandas as pd

# Meta Data

In [17]:
dataset = 'xtreme'
dataset_select_name = 'PAN-X'
langs =['de','fr','it','en']
fracs=[0.629,0.229,0.084,0.059]

In [10]:
dataset_config=get_dataset_config_names(dataset)
dataset_selected=[s for s in dataset_config if s.startswith(dataset_select_name)]

In [22]:
from collections import defaultdict
from datasets import DatasetDict

panx_ch=defaultdict(DatasetDict)

for lang, frac in zip(langs, fracs):
    ds = load_dataset(dataset, name = f"PAN-X.{lang}")
    for split in ds:
        panx_ch[lang][split]=(
            ds[split].shuffle(seed=0)
                     .select(range(int(frac * ds[split].num_rows)))
        )

In [35]:
pd.DataFrame(
    {lang : [panx_ch[lang]['train'].num_rows]for lang in langs},
    index = ['train data 수']    
)

,de,fr,it,en
train data 수,12580,4580,1680,1180


In [43]:
elements=panx_ch['de']['train'][0]
for key,val in elements.items():
    print(f"{key}:{val}")

tokens:['2.000', 'Einwohnern', 'an', 'der', 'Danziger', 'Bucht', 'in', 'der', 'polnischen', 'Woiwodschaft', 'Pommern', '.']
ner_tags:[0, 0, 0, 0, 5, 6, 0, 0, 5, 5, 6, 0]
langs:['de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de']


In [53]:
for k,v in panx_ch['de']['train'].features.items():
    print(f"{k}:{v}")

tokens:Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)
ner_tags:Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None), length=-1, id=None)
langs:Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)


In [56]:
tags=panx_ch['de']['train'].features['ner_tags'].feature
print(tags)

ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None)


In [59]:
def create_tag_name(batch):
    return {"ner_tags_str":[tags.int2str(idx) for idx in batch['ner_tags']]}
panx_de=panx_ch['de'].map(create_tag_name)

Map:   0%|          | 0/12580 [00:00<?, ? examples/s]

Map:   0%|          | 0/6290 [00:00<?, ? examples/s]

Map:   0%|          | 0/6290 [00:00<?, ? examples/s]

In [61]:
def create_tag_len(batch):
    return {"len_token_str":[ len(token) for token in batch['tokens']]}
panx_de_len=panx_de.map(create_tag_len)

Map:   0%|          | 0/12580 [00:00<?, ? examples/s]

Map:   0%|          | 0/6290 [00:00<?, ? examples/s]

Map:   0%|          | 0/6290 [00:00<?, ? examples/s]